In [ ]:
!nvidia-smi

Thu Nov 14 12:04:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Downloading Necessary Libraries

In [1]:
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U datasets
!pip3 install -q -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 19.4 MB/s eta 0:00:00
  

# Hugging Face Logins

In [ ]:
import os
from google.colab import userdata
os.environ["HF_Token"] = userdata.get('HF_Token')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Data Loading and Transformation

In [ ]:
from datasets import load_dataset, DatasetDict

# Load the dataset
dataset = load_dataset("ShenLab/MentalChat16K")

# Split the dataset into train and test
train_test_split = dataset['train'].train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']


train_valid_split = train_dataset.train_test_split(test_size=0.1)
train_dataset = train_valid_split["train"]
validation_dataset = train_valid_split["test"]

# Combine into a DatasetDict
split_dataset = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset
})

print(split_dataset)
dataset = split_dataset


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.58k [00:00<?, ?B/s]

Interview_Data_6K.csv:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

Synthetic_Data_10K.csv:   0%|          | 0.00/32.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16084 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 11580
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1287
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 3217
    })
})


In [ ]:
#function to transform prompt
def transform_example(example):

    instruction = example['instruction'].strip()
    input_text = example['input'].strip() if example['input'] else ""
    output_text = example['output'].strip()


    if input_text:
        formatted_text = f"<s>[INST] {instruction} {input_text} [/INST] {output_text} </s>" #for cases when ther's no INST
    else:
        formatted_text = f"<s>[INST] {instruction} [/INST] {output_text} </s>"

    return {'text': formatted_text}

# transformer function call
transformed_dataset = DatasetDict({
    split: data.map(transform_example, remove_columns=data.column_names)
    for split, data in dataset.items()
})

print(transformed_dataset['train'][0]['text'])

Map:   0%|          | 0/11580 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

Map:   0%|          | 0/3217 [00:00<?, ? examples/s]

<s>[INST] You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description. 
The assistant gives helpful, comprehensive, and appropriate answers to the user's questions. I've been feeling disconnected from my partner and our relationship for a while now. We've been arguing more frequently, and I've been feeling increasingly distant and detached. I'm not sure if I even want to be in this relationship anymore. [/INST] I understand that relationships can be complex and challenging at times, and it's not uncommon to feel disconnected or uncertain about the future. It's important to remember that relationships take work and commitment, and it's essential to communicate openly and honestly with your partner about your feelings and concerns.

Given the complexity of your situation, it might be helpful to consider seeking the support of a mental health professional or couples therapist. They can provide you with the tools and t

# Defining Q-LoRa Parameters

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Downloading pre-trained model and Tokenizer from Hugging Face

In [5]:
#Base model
model_id= "EleutherAI/gpt-neo-2.7B"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=['v_proj', 'up_proj', 'o_proj', 'q_proj', 'down_proj', 'gate_proj', 'k_proj'],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# Training pre-trained model for 1 epoch

In [ ]:
import transformers

from trl import SFTTrainer


tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()


trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset['train'],
    eval_dataset=transformed_dataset['validation'],
    dataset_text_field="text",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        num_train_epochs=1,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=4,
        #warmup_steps=0.03,
        max_steps=-1,
        learning_rate=2e-4,
        logging_steps=25,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        report_to="tensorboard",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/11580 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

# Saving Trained model to Hugging Face Repository

In [ ]:
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
model.config.use_cache = False
trainer.train()


new_model = "GPT-Neox-MentalHealth-Finetune"

trainer.model.save_pretrained(new_model)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


# Push the model and tokenizer to the Hugging Face
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)



/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,1.922400
50,1.550700
75,1.484100


Step,Training Loss
25,1.922400
50,1.550700
75,1.484100
100,1.447000
125,1.402500
150,1.386900
175,1.373700
200,1.361200
225,1.327900
250,1.325200


model-00002-of-00002.safetensors:   0%|          | 0.00/315M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DiatWork/GPT-Neox-MentalHealth-Finetune/commit/2f546cf03edb70a7b1154bc7c478ea3759d4c2ba', commit_message='Upload tokenizer', commit_description='', oid='2f546cf03edb70a7b1154bc7c478ea3759d4c2ba', pr_url=None, pr_revision=None, pr_num=None)

# Saving to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# path in Google Drive to save the model
save_path = "/content/drive/MyDrive/Gpt-Neox-MentalHealth-Finetune"

# directory
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer to Google Drive
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/Gpt-Neox-MentalHealth-Finetune/tokenizer_config.json',
 '/content/drive/MyDrive/Gpt-Neox-MentalHealth-Finetune/special_tokens_map.json',
 '/content/drive/MyDrive/Gpt-Neox-MentalHealth-Finetune/vocab.json',
 '/content/drive/MyDrive/Gpt-Neox-MentalHealth-Finetune/merges.txt',
 '/content/drive/MyDrive/Gpt-Neox-MentalHealth-Finetune/added_tokens.json',
 '/content/drive/MyDrive/Gpt-Neox-MentalHealth-Finetune/tokenizer.json')

# Loading Fine-tuned model from Hugging Face Repo

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import logging
from transformers import logging as transformers_logging
import warnings

logging.basicConfig(level=logging.WARNING)
transformers_logging.set_verbosity_warning()
warnings.filterwarnings("ignore", category=UserWarning)

# Load the model and tokenizer
model_id = "DiatWork/GPT-Neox-MentalHealth-Finetune"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
model.config.use_cache = False  # Disable cache

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/33.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/315M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

# Inference

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
import torch
import textwrap

# Input with Prompt
input_text = "<s>[INST] You are a helpful mental health assistant. I'm very stressed because I have a Job Interview, though I have practiced for it but this is my first interview so I'm anxious. How to deal with this stress? [/INST] "

inputs = tokenizer(input_text, return_tensors="pt").to(model.device). # Tokenize the input

# Generate response
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=400)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip() # Tokenize the output

# Presentation
if "[INST]" in decoded_output and "[/INST]" in decoded_output:
    question, model_answer = decoded_output.split("[/INST]", 1)
else:
    question, model_answer = decoded_output, ""  # Handle cases where split fails


question = "\n".join(textwrap.wrap(question, width=80))
model_answer = "\n".join(textwrap.wrap(model_answer.strip(), width=80))

print("Question:\n", question)
print("\nModel Answer:\n", model_answer)
print("\n" + "=" * 80 + "\n")


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Question:
 <s>[INST] You are a helpful mental health assistant. I'm very stressed because I
have a Job Interview, though I have practiced for it but this is my first
interview so I'm anxious. How to deal with this stress?

Model Answer:
 It's understandable that you're feeling anxious about your upcoming job
interview. It's natural to feel this way when you're preparing for a big event
or when you're facing a significant change in your life.  One strategy that
might help you manage your stress is to focus on the positive aspects of the
situation. Remember that you're not alone in this process and that there are
resources available to support you.  One approach that might be helpful is to
practice relaxation techniques before the interview. Deep breathing exercises,
progressive muscle relaxation, or mindfulness meditation can help you calm down
and prepare for the interview.  Another strategy is to focus on your strengths
and accomplishments. Share these with the interviewer and show th

In [ ]:
# Input with Prompt
input_text = "<s>[INST] You are a helpful mental health assistant. I'm very stressed because I have a group project in last semester of my college and I want to get first honours, I'm scared. Please help me cope with stress. [/INST]"

inputs = tokenizer(input_text, return_tensors="pt").to(model.device). # Tokenize the input

# Generate response
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=200)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip() # Tokenize the output

# Presentation
if "[INST]" in decoded_output and "[/INST]" in decoded_output:
    question, model_answer = decoded_output.split("[/INST]", 1)
else:
    question, model_answer = decoded_output, ""  # Handle cases where split fails


question = "\n".join(textwrap.wrap(question, width=80))
model_answer = "\n".join(textwrap.wrap(model_answer.strip(), width=80))

print("Question:\n", question)
print("\nModel Answer:\n", model_answer)
print("\n" + "=" * 80 + "\n")



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Question:
 <s>[INST] You are a helpful mental health assistant. I'm very stressed because I
have a group project in last semester of my college and I want to get first
honours, I'm scared. Please help me cope with stress.

Model Answer:
 It's understandable that you're feeling stressed about your group project. It's
natural to want to get first honours, but it's important to remember that it's
not a guarantee of success. It's common for students to feel anxious about their
group projects, but it's important to remember that it's not a reflection of
your abilities or intelligence. Instead, it's a reflection of your commitment
and effort.  One strategy that might help you cope with stress is to focus on
the positive aspects of your group project. Remember that you're not alone in
this, and there are many people who have faced similar challenges. Focus on the
positive aspects of your group project, such as the people you're working with,
the assignments you're working on, and the feedback

# Evaluation

In [ ]:
!pip install evaluate bert_score transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.8 MB/s eta 0:00:00


# Validation set Response under max_length=100

In [ ]:
import textwrap
import torch
import evaluate

# validation data for evaluation
test_data = dataset["test"]
subset_size = len(test_data)

prompts = []
references = []

# Build prompts and references
for i in range(subset_size):
    sample = test_data[i]
    instruction = sample['instruction']
    input_text = sample['input']
    output_text = sample['output']

    if input_text:
        prompt = f"<s>[INST] {instruction} {input_text} [/INST]"
    else:
        prompt = f"<s>[INST] {instruction} [/INST]"

    prompts.append(prompt)
    references.append(output_text)  # reference answer

# Generate responses for each prompt and format them
generated_texts = []
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=True,
            top_p=0.9,
            top_k=50,
            temperature=0.7,
            use_cache=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the response
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # Presentation
    try:
        question, model_answer = decoded_output.split("[/INST]", 1)
    except ValueError:
        question, model_answer = decoded_output, ""


    question = "\n".join(textwrap.wrap(question, width=80))
    model_answer = "\n".join(textwrap.wrap(model_answer, width=80))

    print("Question:\n", question)
    print("\nModel Answer:\n", model_answer)
    print("\n" + "="*80 + "\n")

    # Append decoded output to generated_texts for evaluation
    generated_texts.append(decoded_output)




Streaming output truncated to the last 5000 lines.
negative thought patterns and associated automatic thoughts. Through this
process, you can learn to reframe anxious thoughts and refocus your attention on
more positive and realistic thoughts.  Another technique that can be


Question:
 <s>[INST] You are a helpful mental health counselling assistant, please answer
the mental health questions based on the patient's description.  The assistant
gives helpful, comprehensive, and appropriate answers to the user's questions.
I've been struggling with understanding my own sexuality and it's causing me a
lot of confusion and frustration. Society has these expectations and labels that
make it hard for me to fully embrace who I am. I feel like I have to hide a part
of myself because I fear judgment and rejection. How do I navigate through this
journey of self-discovery without losing my sense of identity?

Model Answer:
  It's important to remember that everyone's sexuality is different, and it'

# Testing on 50% Test Data

In [ ]:
dataset['test']

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 3217
})

In [ ]:
import textwrap
import torch
import evaluate

# 50% test data for evaluation
test_data = dataset["test"]
subset_size = int(0.5 * len(test_data))

prompts = []
references = []

# Build prompts and references
for i in range(subset_size):
    sample = test_data[i]
    instruction = sample['instruction']
    input_text = sample['input']
    output_text = sample['output']

    if input_text:
        prompt = f"<s>[INST] {instruction} {input_text} [/INST]"
    else:
        prompt = f"<s>[INST] {instruction} [/INST]"

    prompts.append(prompt)
    references.append(output_text)  # reference answer

# Generate responses for each prompt and format them
generated_texts = []
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=True,
            top_p=0.9,
            top_k=50,
            temperature=0.7,
            use_cache=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the response
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # Presentation
    try:
        question, model_answer = decoded_output.split("[/INST]", 1)
    except ValueError:
        question, model_answer = decoded_output, ""


    question = "\n".join(textwrap.wrap(question, width=80))
    model_answer = "\n".join(textwrap.wrap(model_answer, width=80))

    print("Question:\n", question)
    print("\nModel Answer:\n", model_answer)
    print("\n" + "="*80 + "\n")

    # Append decoded output to generated_texts for evaluation
    generated_texts.append(decoded_output)



!pip install rouge_score
# Initialize evaluation metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

# Calculate Scores
bleu_score = bleu.compute(predictions=generated_texts, references=[[ref] for ref in references])
rouge_score = rouge.compute(predictions=generated_texts, references=references)
bert_score = bertscore.compute(predictions=generated_texts, references=references, lang="en")

# Print Results
print("BLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)
print("BERTScore (Precision, Recall, F1):", bert_score["precision"], bert_score["recall"], bert_score["f1"])


from huggingface_hub import Repository

repo = Repository(local_dir="./evaluation_results", clone_from="DiatWork/GPT-Neox-MentalHealth-Finetune-10epoch")

# Save metrics to JSON file
evaluation_metrics = {
    "ROUGE": rouge_score,
    "BERTScore": bert_score,
    "BLEU": bleu_score,
}

import json

output_file = "evaluation_metrics.json"
with open(output_file, "w") as json_file:
    json.dump(evaluation_metrics, json_file, indent=4)



Streaming output truncated to the last 5000 lines.
These activities can help distract you from your grief and provide a sense of
calmness and well-


Question:
 <s>[INST] You are a helpful mental health counselling assistant, please answer
the mental health questions based on the patient's description.  The assistant
gives helpful, comprehensive, and appropriate answers to the user's questions.
I'm worried that I can't remember the exact details of our last trip to Hawaii.
I know my husband booked a condo for us, but I can't recall the dates or if we
went before or after the [Name]'s concert.

Model Answer:
  It's natural to feel a bit anxious about your memory loss, especially when it
comes to the details of our trip to Hawaii. The best way to address this is to
create a detailed record of our trip, including the details of the event, the
names of our friends and family, and any special events that may have occurred.
This information can be helpful during future trips. </s> </p>  The 

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/path/to/your/local/model/dir is already a clone of https://huggingface.co/DiatWork/GPT-Neox-MentalHealth-Finetune. Make sure you pull the latest changes with `repo.git_pull()`.


BLEU Score: {'bleu': 0.06995359388760698, 'precisions': [0.47826164989423753, 0.1456347173269965, 0.06309832501312017, 0.0347506886015353], 'brevity_penalty': 0.62926286215011, 'length_ratio': 0.6834306721547889, 'translation_length': 390025, 'reference_length': 570687}
ROUGE Score: {'rouge1': 0.3828811403800765, 'rouge2': 0.11400396058279869, 'rougeL': 0.17227985362619608, 'rougeLsum': 0.30002664783280414}
BERTScore (Precision, Recall, F1): [0.8546341061592102, 0.8714326620101929, 0.8668084144592285, 0.8384053111076355, 0.8599699139595032, 0.8550359606742859, 0.8594415187835693, 0.8552557826042175, 0.8645744323730469, 0.8621082305908203, 0.8534358739852905, 0.8481904864311218, 0.8580842614173889, 0.8562780618667603, 0.8461532592773438, 0.854697048664093, 0.868019700050354, 0.8571382761001587, 0.8523473739624023, 0.8564121723175049, 0.8361924886703491, 0.8575438857078552, 0.8304237723350525, 0.833426833152771, 0.8603389263153076, 0.862920343875885, 0.8499051332473755, 0.842072606086731

OSError: On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


# Evaluation Results

In [ ]:
import json
import numpy as np

file_path = '/content/evaluation_metrics.json'

with open(file_path, 'r') as file:
    gemma_data = json.load(file)

# Calculate Avg. BERTScore
bert_score = gemma_data["BERTScore"]

precision_avg = np.mean(bert_score["precision"])
recall_avg = np.mean(bert_score["recall"])
f1_avg = np.mean(bert_score["f1"])

precision_avg, recall_avg, f1_avg


(0.8564294119601819, 0.8514925939452589, 0.8538805755290819)